In [1]:
import os,sys,inspect

import pnsolver

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

import util
import stencil
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable

import ipywidgets as widgets
from ipywidgets.widgets.interaction import interact

import scipy.io

 # Use interactive matplotlib backend, which runs in the browser
%matplotlib notebook

In [ ]:
def load_pn_solution(filename, silent = False):
    data = util.load_pn_system(filename, silent=silent)
    staggered = False
    pni = stencil.PNInfo2D( data["order"], staggered )
    if data["resolution"].shape[0] == 2:
        data["resolution"] = np.array([data["resolution"][0], data["resolution"][1], 1])
    domain = pnsolver.Domain( np.array([7.0, 7.0, 1.0]), data["resolution"], np.array([0.0, 0.0, 0.0]) )
    data["domain"] = domain
    data["pni"] = pni
    
    if "x" in data:
        x_complex = pni.to_complex(data["x"])
    else:
        data["x"] = np.zeros(data["b"].shape)
        x_complex = np.zeros(data["b"].shape, dtype=complex)

    # contruct radiance field from coefficients
    L = pnsolver.SHEXP( data["order"] )
    for index in range(data["numCoeffs"]):
        (l,m) = pni.lm_index(index)
        sh_index = util.sh_index(l,m)
        offset = pni.getOffset(index)*0.5


        u0 = util.extract_coefficient_field( x_complex, data["resolution"], data["numCoeffs"], index )
        L.setCoefficientField( l, m, pnsolver.VoxelGridField( u0, domain, offset ) )
        #print(u0.dtype)
    data["L"] = L
    return data

def load_pn_solution3d(filename, silent = False):
    data = util.load_pn_system(filename, silent=silent)
    staggered = False
    pni = stencil.PNInfo3D( data["order"], staggered )
    domain = pnsolver.Domain( np.array([7.0, 7.0, 7.0]), data["resolution"], np.array([0.0, 0.0, 0.0]) )
    data["domain"] = domain
    data["pni"] = pni
    
    if "x" in data:
        x_complex = pni.to_complex(data["x"])
    else:
        data["x"] = np.zeros(data["b"].shape)
        x_complex = np.zeros(data["b"].shape, dtype=complex)

    # contruct radiance field from coefficients
    L = pnsolver.SHEXP( data["order"] )
    for index in range(data["numCoeffs"]):
        (l,m) = pni.lm_index(index)
        sh_index = util.sh_index(l,m)
        offset = pni.getOffset(index)*0.5


        u0 = util.extract_coefficient_field( x_complex, data["resolution"], data["numCoeffs"], index )
        L.setCoefficientField( l, m, pnsolver.VoxelGridField( u0, domain, offset ) )
        #print(u0.dtype)
    data["L"] = L
    return data

def render(L, pWS):
    plot_y = np.arange(0, 2, 0.01)
    plot_x = 2 * np.pi * plot_y

    for i in range(plot_x.shape[0]):
        phi = plot_x[i]
        omega = util.sphericalDirection(np.pi*0.5, phi)
        # for some reason (I still have to investigate) the direction seems to be inverted
        # for our solution
        #omega = -omega
        plot_y[i] = np.real(L.eval(pWS, np.array([omega[0], omega[1]])))
    return (plot_x, plot_y)

In [ ]:
datasets = []

datasets.append(("from id", "---"))
#datasets.append(("checkerboard fopn p1", "C:/projects/epfl/epfl17/python/pnsolver/results/studies/checkerboard_fopn_p1.mat"))
#datasets.append(("checkerboard fopn p2", "C:/projects/epfl/epfl17/python/pnsolver/results/studies/checkerboard_fopn_p2.mat"))
#datasets.append(("checkerboard fopn p3", "C:/projects/epfl/epfl17/python/pnsolver/results/studies/checkerboard_fopn_p3.mat"))
#datasets.append(("checkerboard fopn p4", "C:/projects/epfl/epfl17/python/pnsolver/results/studies/checkerboard_fopn_p4.mat"))
#datasets.append(("checkerboard fopn p5", "C:/projects/epfl/epfl17/python/pnsolver/results/studies/checkerboard_fopn_p5.mat"))
#datasets.append(("checkerboard sopn p1", "C:/projects/epfl/epfl17/python/pnsolver/results/studies/checkerboard_sopn_p1.mat"))
#filename = "C:/projects/epfl/epfl17/python/pnsolver/results/studies/vacuum_sopn_p1.mat"
#filename = "C:/projects/epfl/epfl17/python/pnsolver/results/studies/checkerboard_sopn_p1_vacuum.mat"
#filename = "C:/projects/epfl/epfl17/python/notebooks/sopn/solution_checkerboard_blur10.0_term1.mat"

In [ ]:
def make_plot():
    global msg
    figsize = plt.figaspect(1/2)
    #fig, ax = plt.subplots(1, 2, figsize=figsize)
    
    fig = plt.figure(figsize=figsize);
    ax = [None, None, None]
    ax[0] = fig.add_subplot(121)
    #ax[1] = fig.add_subplot(122)
    ax[1] = fig.add_subplot(122, projection='polar')
    #ax[2] = fig.add_subplot(133)

    
    # Plot 1 =============================================
    # Draw an empty image
    img_view = ax[0].imshow(np.zeros((1,1)), cmap='jet', origin='lower')
    
    # Add a nice color bar
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    divider = make_axes_locatable(ax[0])
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(img_view, cax=cax)

    # Plot 2 =============================================
    # Add an empty plot of the SH coefficients ------
    plot_view = ax[1].plot(np.zeros((1)))[0]
    ax[1].set(adjustable="datalim")
    ax[1].set_rmax(1.0)
    #plot_view = ax[1].imshow(np.zeros((1,1)), cmap='jet', origin='lower')
    

    result = { 'img_view' : img_view,
               'img_ax' : ax[0],
               'plot_view' : plot_view,
               'plot_ax' : ax[1],
               #'img_view2':img_view2,
               #'img_ax2':ax[2],
               'dataset_filename':None
             }
    
    def on_click(event):
        global msg
        if event.inaxes is not ax[0] or 'on_click_handler' not in result:
            return
        result['on_click_handler'](np.array([event.ydata, event.xdata]))

    fig.canvas.mpl_connect('button_press_event', on_click)
    fig.tight_layout()

    return result

def plot_interact(plot):
    def redraw(dataset_id, coeff_index, display_absolute_value):
        img_view = plot['img_view']
        plot_view = plot['plot_view']
        plot_ax = plot['plot_ax']
        
        dataset_filename = "C:/projects/epfl/epfl17/python/pnsolver/results/studies/checkerboard{}.mat".format(dataset_id)
        if dataset_filename != plot['dataset_filename']:
            plot['dataset_filename'] = dataset_filename
            plot["data"] = load_pn_solution(dataset_filename)

        data = plot["data"]
        domain = data["domain"]
        pni = data["pni"]
        L = data["L"]
                    
        #filename = "C:/projects/epfl/epfl17/python/pnsolver/results/studies/checkerboard_groundtruth.mat"
        #groundtruth_image = util.load_pn_system(filename)["x"]
        #groundtruth_image = np.clip(image, 1.0e-8, np.max(image))
        #plot_view.set_norm(LogNorm(vmin=np.min(groundtruth_image), vmax=np.max(groundtruth_image)))
        #plot_view.set_extent((domain.getBoundMin()[0], domain.getBoundMax()[0], domain.getBoundMin()[1], domain.getBoundMax()[1]))
        #plot_view.set_data(groundtruth_image.T)

            

        
        # update plot 1 =======================================
        u = util.extract_coefficient_field( data["x"], data["resolution"], data["numCoeffs"], coeff_index )
        
        if display_absolute_value == True:
            u = np.abs( u)
            vmax = np.max(u)
        else:
            u = np.clip( u, 1.0e-8, np.max(u) )
            #vmax = np.max(u)
            vmax = 1.3
        #u = np.abs(u)
        #print( "min={} max={}".format(np.min(u), np.max(u)) )
        
        # Update the plot
        img_view.set_data(u)
        #img_view.set_clim(vmin=0, vmax=np.percentile(data_slice, 99))
        #img_view.set_clim(vmin=np.min(u), vmax=np.max(u))
        img_view.set_norm(LogNorm(vmin=np.min(u), vmax=vmax))
        img_view.set_extent((domain.getBoundMin()[0], domain.getBoundMax()[0], domain.getBoundMin()[1], domain.getBoundMax()[1]))
        
        # update plot 2 =======================================
        def on_click_handler(pos):
            pWS = np.array([pos[1], pos[0]])
            (plot_x, plot_y) = render(L, pWS)
            plot_y = plot_y/np.max(plot_y)
            plot_view.set_data(plot_x, plot_y)
        
        plot['on_click_handler'] = on_click_handler

    #slice_index_widget = widgets.IntSlider(
    #    min=0,
    #    max=127,
    #    value=64,
    #    description='Slice'
    #)
    
    dataset_id_widget = widgets.Text(
        value='_fopn_p1_sg',
        placeholder='Type something',
        description='String:',
        disabled=False
    )
    
    #rightside_index_widget = widgets.Dropdown(
    #    options=[("L", 0), ("groundtruth", 1)],
    #    value=0,
    #    description='right side'
    #)
   
    coeff_index_widget = widgets.IntSlider(
        min=0,
        max=2,
        value=0,
        description='coefficient'
    )
    
    display_absolute_value_widget = widgets.ToggleButton(
    value=False,
    description='absolute',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
    )
    
    
    #interact(redraw, dataset_index=dataset_index_widget, slice_index=slice_index_widget, coeff_index=coeff_index_widget)
    interact(redraw, dataset_id=dataset_id_widget, coeff_index=coeff_index_widget, display_absolute_value = display_absolute_value_widget)

In [ ]:
plot = make_plot()

In [ ]:
plot_interact(plot)

In [ ]:
filename = "C:/projects/epfl/epfl17/python/pnsolver/results/studies/checkerboard_sopn_p1_sg.mat"
#filename = "C:/projects/epfl/epfl17/python/pnsolver/results/studies/vacuum_sopn_p1.mat"
#filename = "C:/projects/epfl/epfl17/python/pnsolver/results/studies/checkerboard_sopn_p1_vacuum.mat"
#filename = "C:/projects/epfl/epfl17/python/notebooks/sopn/solution_checkerboard_blur10.0_term1.mat"

In [ ]:
data = load_pn_solution(filename)
domain = data["domain"]
L = data["L"]

In [ ]:
#rows = np.arange(0, 7.0, 0.1)

#print(plot_x.shape)

(plot_x,plot_y) = render(data["L"], np.array([3.5, 3.5]))

res_x = 70

image = np.zeros((res_x, plot_x.shape[0]))

for i in range(res_x):
    pVS = np.array([i+0.5, 35])
    #pWS = domain.voxelToWorld( pVS )
    pWS = np.array([3.5, 1.5])
    (plot_x,plot_y) = render(data["L"], pWS)
    image[i, :] = plot_y
    
image = np.clip(image, 1.0e-8, np.max(image))
#image = np.abs(image)
    
fig = plt.figure(figsize=(8,8));
ax = fig.add_subplot(111)
img_view = ax.imshow(image, cmap='jet', norm=LogNorm(vmin=np.min(image), vmax=np.max(image)), origin='lower')
plt.show()

In [ ]:

res_restricted = np.array([data["resolution"][0]//2, data["resolution"][1]//2, 1])
print(res_restricted)

In [ ]:
filename = "C:/projects/epfl/epfl17/python/pnsolver/results/studies/checkerboard_test.mat"
data = util.load_pn_system(filename)
image = util.extract_coefficient_field( data["x"], data["resolution"], data["numCoeffs"], 0 )
#image = util.extract_coefficient_field( data["x"], res_restricted, data["numCoeffs"], 2 )

print("image min={} max={}".format(np.min(image), np.max(image)))
'''
image_diff = np.zeros((71, 71))

for i in range(35):
    for j in range(71):
        image_diff[i, j] = np.abs(image[i, j, 0] - image[71 - i - 1, j, 0])
        #if i ==21 and j == 38:
        #    image_diff[i, j] = 1.0
print(np.max(image_diff))
'''
image = np.clip(image, 1.0e-8, np.max(image))



fig = plt.figure(figsize=(10,10));
ax = fig.add_subplot(111)
img_view = ax.imshow(image[:,:,0], cmap='jet', norm=LogNorm(vmin=np.min(image), vmax=np.max(image)), origin='lower')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(img_view, cax=cax)

#ax = fig.add_subplot(122)
#img_view = ax.imshow(image_diff, cmap='jet', vmin=np.min(image_diff), vmax=np.max(image_diff), origin='lower')
#divider = make_axes_locatable(ax)
#cax = divider.append_axes("right", size="5%", pad=0.05)
#plt.colorbar(img_view, cax=cax)

plt.show()

In [ ]:
convergence_cg = data["convergence"].T
convergence_time_cg = data["timestamps"].T

In [ ]:
convergence_mg = data["convergence"].T
convergence_time_mg = data["timestamps"].T

In [ ]:
convergence_gs = data["convergence"].T
convergence_time_gs = data["timestamps"].T

In [ ]:
fig = plt.figure(figsize=(10,10));

a = np.linspace(0.0, 1.0)
#plt.semilogy( convergence_cg, label="CG" )
#plt.semilogy( convergence_time_cg, convergence_cg, label="CG" )
plt.loglog( convergence_time_cg, convergence_cg, label="CG" )
#plt.semilogy( convergence_mg, label="Multigrid" )
#plt.semilogy( convergence_time_mg, convergence_mg, label="Multigrid" )
plt.loglog( convergence_time_mg, convergence_mg, label="Multigrid" )
#plt.loglog( convergence_mg, label="Multigrid" )
#plt.semilogy( convergence_time_gs, convergence_gs, label="Multigrid2" )
#plt.plot( convergence_mg, label="Multigrid" )
plt.legend(loc='best')
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,10));

a = np.linspace(0.0, 1.0)
plt.semilogy( data["convergence"].T, label="CG" )
plt.legend(loc='best')
plt.show()

In [ ]:
image = util.extract_coefficient_field( data["x"], data["resolution"], data["numCoeffs"], 0 )


image_diff = np.zeros((71, 71))
for i in range(35):
    for j in range(71):
        image_diff[i, j] = np.abs(image[i, j, 0] - image[71 - i - 1, j, 0])
print(np.max(image_diff))

#image = data["x"]
#image = np.clip(image, 1.0e-8, np.max(image))
#print(data["x"].dtype)
#print(np.min(image))
#print(np.max(image))


fig = plt.figure(figsize=(8,8));
ax = fig.add_subplot(111)
#img_view = ax.imshow(image[:,:,0], cmap='jet', vmin=np.min(image), vmax=np.max(image), origin='lower')
img_view = ax.imshow(image_diff, cmap='jet', vmin=np.min(image), vmax=np.max(image), origin='lower')
#img_view = ax.imshow(image, cmap='jet', vmin=np.min(image), vmax=np.max(image), origin='lower')

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(img_view, cax=cax)

plt.show()

-------------------------------


# 3D Solver

In [ ]:
filename = "C:/projects/epfl/epfl17/python/pnsolver/results/studies/pointsource_test.mat"
data = util.load_pn_system(filename)
image = extract_coefficient_field( data["x"].T, data["resolution"], data["numCoeffs"], 0 )
#image = data["x"]
#image = np.clip(image, 1.0e-8, np.max(image))
#print(data["x"].dtype)
#print(np.min(image))
#print(np.max(image))

filename = "C:/projects/epfl/epfl17/python/pnsolver/results/studies/pointsource_test.mat"
data = util.load_pn_system(filename)
image_cda = extract_coefficient_field( data["x"].T, data["resolution"], data["numCoeffs"], 0 )




In [ ]:



def grosjean(xi, albedo):
    diffusion_exponent = np.sqrt(3.0*(1.0-albedo)/(2.0-albedo))
    factor = sigma_t*sigma_t/(4.0*np.pi)
    return factor*(np.exp(-xi)/(xi*xi) + 3.0*albedo/((2.0-albedo)*xi) * np.exp(-diffusion_exponent*xi))
def cda(xi, albedo):
    diffusion_exponent = np.sqrt(3.0*(1.0-albedo))
    factor = sigma_t*sigma_t/(4.0*np.pi)
    return factor * (3.0/xi) * np.exp(-diffusion_exponent*xi)
    




#x_list = np.array([domain.voxelToWorld( np.array([i, center_voxel[1], center_voxel[2]]) )[0] for i in range(res)])
#x_list_highres = np.arange(0.0, size, 100)
#pointsource_grosjean = np.array([grosjean(np.linalg.norm(x-pointsource_center)*sigma_t, albedo) for x in x_list])
#pointsource_cda = np.array([cda(np.linalg.norm(x-pointsource_center)*sigma_t, albedo) for x in x_list])


#fig = plt.figure(figsize=(8,8));
#ax = fig.add_subplot(111)
#plt.semilogy(x_list, image_cda[:,center_voxel[1],center_voxel[2]], label="CDA")
#plt.semilogy(x_list, image_p1[:,center_voxel[1],center_voxel[2]], label="P1", marker=".", linestyle=" ")
#plt.semilogy(x_list, pointsource_grosjean, label="Grosjean")
#plt.semilogy(x_list, pointsource_cda, label="CDA")
#plt.legend()
#plt.show()

In [ ]:
sigma_t = 8.0
albedo = 0.9
size = 7.0


In [2]:

filename = "C:/projects/epfl/epfl17/python/pnsolver/results/studies/pointsource_stencil_cda.mat"
data = util.load_pn_system(filename)
image_cda = util.extract_coefficient_field( data["x"].T, data["resolution"], data["numCoeffs"], 0 )

filename = "C:/projects/epfl/epfl17/python/pnsolver/results/studies/pointsource_stencil_fopn_p1_sg.mat"
data = util.load_pn_system(filename)
image_p1 = util.extract_coefficient_field( data["x"].T, data["resolution"], data["numCoeffs"], 0 )

filename = "C:/projects/epfl/epfl17/python/pnsolver/results/studies/pointsource_stencil_fopn_p2_sg.mat"
data = util.load_pn_system(filename)
image_p2 = util.extract_coefficient_field( data["x"].T, data["resolution"], data["numCoeffs"], 0 )

filename = "C:/projects/epfl/epfl17/python/pnsolver/results/studies/pointsource_stencil_fopn_p3_sg.mat"
data = util.load_pn_system(filename)
image_p3 = util.extract_coefficient_field( data["x"].T, data["resolution"], data["numCoeffs"], 0 )

filename = "C:/projects/epfl/epfl17/python/pnsolver/results/studies/pointsource_stencil_fopn_p4_sg.mat"
data = util.load_pn_system(filename)
image_p4 = util.extract_coefficient_field( data["x"].T, data["resolution"], data["numCoeffs"], 0 )

filename = "C:/projects/epfl/epfl17/python/pnsolver/results/studies/pointsource_stencil_fopn_p4_sg2.mat"
data = util.load_pn_system(filename)
image_p4_2 = util.extract_coefficient_field( data["x"].T, data["resolution"], data["numCoeffs"], 0 )

size = 7.0
res = data["resolution"][0]
domain = pnsolver.Domain( np.array([size, size, size]), np.array([res, res, res]), np.array([0.0, 0.0, 0.0]))
center_voxel = np.array([int(res/2), int(res/2), int(res/2)])
pointsource_center = domain.voxelToWorld( np.array([center_voxel[0]+0.5, center_voxel[1]+0.5, center_voxel[2]+0.5]) )


loading PN solution from C:/projects/epfl/epfl17/python/pnsolver/results/studies/pointsource_stencil_cda.mat
	order=1  numCoeffs=1  resolution=50 50
loading PN solution from C:/projects/epfl/epfl17/python/pnsolver/results/studies/pointsource_stencil_fopn_p1_sg.mat
	order=1  numCoeffs=4  resolution=50 50
loading PN solution from C:/projects/epfl/epfl17/python/pnsolver/results/studies/pointsource_stencil_fopn_p2_sg.mat
	order=2  numCoeffs=9  resolution=50 50
loading PN solution from C:/projects/epfl/epfl17/python/pnsolver/results/studies/pointsource_stencil_fopn_p3_sg.mat
	order=3  numCoeffs=16  resolution=50 50
loading PN solution from C:/projects/epfl/epfl17/python/pnsolver/results/studies/pointsource_stencil_fopn_p4_sg.mat
	order=4  numCoeffs=25  resolution=50 50
loading PN solution from C:/projects/epfl/epfl17/python/pnsolver/results/studies/pointsource_stencil_fopn_p4_sg2.mat
	order=4  numCoeffs=25  resolution=64 64


In [ ]:
def phi_normalized(phi, sigma_t):
    return 4.0*np.pi*phi/(sigma_t*sigma_t)


def cda_org( r, sigma_t, albedo ):
    sigma_a = (1.0-albedo)*sigma_t
    sigma_tr = np.sqrt(3.0*sigma_a*sigma_t)
    return 3.0*sigma_t*np.exp(-sigma_tr*r)/(4.0*np.pi*r)

solution_cda = pnsolver.VoxelGridField( image_cda.astype(complex), domain, np.array([0.5, 0.5, 0.5]) )
#solution_p1 = pnsolver.VoxelGridField( image_p1.astype(complex), domain, np.array([0.5, 0.5, 0.5]) )
#solution_p2 = pnsolver.VoxelGridField( image_p2.astype(complex), domain, np.array([0.5, 0.5, 0.5]) )
#solution_p3 = pnsolver.VoxelGridField( image_p3.astype(complex), domain, np.array([0.5, 0.5, 0.5]) )
#solution_p4 = pnsolver.VoxelGridField( image_p4.astype(complex), domain, np.array([0.5, 0.5, 0.5]) )
#solution_p4_2 = pnsolver.VoxelGridField( image_p4_2.astype(complex), domain, np.array([0.5, 0.5, 0.5]) )

#x_list = np.array([domain.voxelToWorld( np.array([i, center_voxel[1], center_voxel[2]]) )[0] for i in range(res)])
r_list = np.linspace(1.0e-2, 3.0, 100)
#fluece_cda = [ cda_org(r, sigma_t, albedo) for r in r_test ]
#fluece_cda_numerical = [ np.real(solution_cda(pointsource_center+np.array([r, 0.0, 0.0]))) for r in r_list ]
#fluece_p1_numerical = [ np.real(solution_p1(pointsource_center+np.array([r, 0.0, 0.0]))) for r in r_list ]
#fluece_p2_numerical = [ np.real(solution_p2(pointsource_center+np.array([r, 0.0, 0.0]))) for r in r_list ]
#fluece_p3_numerical = [ np.real(solution_p3(pointsource_center+np.array([r, 0.0, 0.0]))) for r in r_list ]
#fluece_p4_numerical = [ np.real(solution_p4(pointsource_center+np.array([r, 0.0, 0.0]))) for r in r_list ]
#fluece_p4_2_numerical = [ np.real(solution_p4_2(pointsource_center+np.array([r, 0.0, 0.0]))) for r in r_list ]

fig = plt.figure(figsize=(8,8));
ax = fig.add_subplot(111)
plt.loglog(r_list, fluece_cda, label="CDA")
#plt.loglog(r_list, fluece_cda_numerical, label="CDA numerical", marker=".", linestyle=" " )
#plt.loglog(r_list, fluece_p1_numerical, label="P1", marker=" ", linestyle="-" )
#plt.loglog(r_list, fluece_p2_numerical, label="P2", marker=" ", linestyle="-" )
#plt.loglog(r_list, fluece_p3_numerical, label="P3", marker=" ", linestyle="-" )
#plt.loglog(r_list, fluece_p4_numerical, label="P4", marker=" ", linestyle="-" )
#plt.loglog(r_list, fluece_p4_2_numerical, label="P4_2", marker=" ", linestyle="-" )
plt.legend()
plt.show()




In [ ]:

print(r_test)

In [ ]:
fig = plt.figure(figsize=(8,8));
ax = fig.add_subplot(111)
img_view = ax.imshow(image[:,:,10].T, cmap='jet', norm=LogNorm(vmin=np.min(image), vmax=np.max(image)), origin='lower')
#img_view = ax.imshow(image, cmap='jet', vmin=np.min(image), vmax=np.max(image), origin='lower')

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(img_view, cax=cax)

plt.show()

In [ ]:
image = image_p1

In [ ]:
def make_plot():
    global msg
    figsize = plt.figaspect(1/2)
    #fig, ax = plt.subplots(1, 2, figsize=figsize)
    
    fig = plt.figure(figsize=figsize);
    ax = [None, None, None]
    ax[0] = fig.add_subplot(121)
    #ax[1] = fig.add_subplot(122)
    ax[1] = fig.add_subplot(122, projection='polar')
    #ax[2] = fig.add_subplot(133)

    
    # Plot 1 =============================================
    # Draw an empty image
    img_view = ax[0].imshow(np.zeros((1,1)), cmap='jet', origin='lower')
    
    # Add a nice color bar
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    divider = make_axes_locatable(ax[0])
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(img_view, cax=cax)

    # Plot 2 =============================================
    # Add an empty plot of the SH coefficients ------
    plot_view = ax[1].plot(np.zeros((1)))[0]
    ax[1].set(adjustable="datalim")
    ax[1].set_rmax(1.0)
    #plot_view = ax[1].imshow(np.zeros((1,1)), cmap='jet', origin='lower')
    

    result = { 'img_view' : img_view,
               'img_ax' : ax[0],
               'plot_view' : plot_view,
               'plot_ax' : ax[1],
               #'img_view2':img_view2,
               #'img_ax2':ax[2],
               'dataset_filename':None
             }
    
    def on_click(event):
        global msg
        if event.inaxes is not ax[0] or 'on_click_handler' not in result:
            return
        result['on_click_handler'](np.array([event.ydata, event.xdata]))

    fig.canvas.mpl_connect('button_press_event', on_click)
    fig.tight_layout()

    return result

def plot_interact(plot):
    def redraw(dataset_id, coeff_index, z_slice, display_absolute_value):
        img_view = plot['img_view']
        plot_view = plot['plot_view']
        plot_ax = plot['plot_ax']
        
        #dataset_filename = "C:/projects/epfl/epfl17/python/pnsolver/results/studies/checkerboard{}.mat".format(dataset_id)
        #if dataset_filename != plot['dataset_filename']:
        #    plot['dataset_filename'] = dataset_filename
        #    plot["data"] = load_pn_solution(dataset_filename)
        #plot["data"] = data
        #data = plot["data"]
        #domain = data["domain"]
        #pni = data["pni"]
        #L = data["L"]
                    
        #filename = "C:/projects/epfl/epfl17/python/pnsolver/results/studies/checkerboard_groundtruth.mat"
        #groundtruth_image = util.load_pn_system(filename)["x"]
        #groundtruth_image = np.clip(image, 1.0e-8, np.max(image))
        #plot_view.set_norm(LogNorm(vmin=np.min(groundtruth_image), vmax=np.max(groundtruth_image)))
        #plot_view.set_extent((domain.getBoundMin()[0], domain.getBoundMax()[0], domain.getBoundMin()[1], domain.getBoundMax()[1]))
        #plot_view.set_data(groundtruth_image.T)

            

        
        # update plot 1 =======================================
        #u = util.extract_coefficient_field( data["x"], data["resolution"], data["numCoeffs"], coeff_index )
        res_z = 35
        u = image[:,:,int(z_slice*res_z)]
        
        if display_absolute_value == True:
            u = np.abs( u)
            vmax = np.max(u)
        else:
            u = np.clip( u, 1.0e-8, np.max(u) )
            vmax = np.max(u)
            #vmax = 1.3
        #u = np.abs(u)
        #print( "min={} max={}".format(np.min(u), np.max(u)) )
        
        # Update the plot
        img_view.set_data(u)
        #img_view.set_clim(vmin=0, vmax=np.percentile(data_slice, 99))
        #img_view.set_clim(vmin=np.min(u), vmax=np.max(u))
        img_view.set_norm(LogNorm(vmin=np.min(u), vmax=vmax))
        #img_view.set_extent((domain.getBoundMin()[0], domain.getBoundMax()[0], domain.getBoundMin()[1], domain.getBoundMax()[1]))
        
        # update plot 2 =======================================
        def on_click_handler(pos):
            pWS = np.array([pos[1], pos[0]])
            (plot_x, plot_y) = render(L, pWS)
            #plot_y = plot_y/np.max(plot_y)
            #plot_view.set_data(plot_x, plot_y)
        
        plot['on_click_handler'] = on_click_handler

    #slice_index_widget = widgets.IntSlider(
    #    min=0,
    #    max=127,
    #    value=64,
    #    description='Slice'
    #)
    
    dataset_id_widget = widgets.Text(
        value='_fopn_p1_sg',
        placeholder='Type something',
        description='String:',
        disabled=False
    )
    
    #rightside_index_widget = widgets.Dropdown(
    #    options=[("L", 0), ("groundtruth", 1)],
    #    value=0,
    #    description='right side'
    #)
   
    coeff_index_widget = widgets.IntSlider(
        min=0,
        max=2,
        value=0,
        description='coefficient'
    )

    z_slice_widget = widgets.FloatSlider(
        min=0.0,
        max=1.0,
        value=0,
        description='z-slice'
    )

    display_absolute_value_widget = widgets.ToggleButton(
    value=False,
    description='absolute',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
    )
    
    
    #interact(redraw, dataset_index=dataset_index_widget, slice_index=slice_index_widget, coeff_index=coeff_index_widget)
    interact(redraw, dataset_id=dataset_id_widget, coeff_index=coeff_index_widget, z_slice=z_slice_widget, display_absolute_value = display_absolute_value_widget)

In [ ]:
plot = make_plot()

In [ ]:
plot_interact(plot)

In [ ]:
print(data)

--------------------

In [ ]:
path = "C:/projects/epfl/epfl17/python/pnsolver/results/studies"
filename = path+"/checkerboard_problem.mat"
data = scipy.io.loadmat(filename)
sigma_t = data["sigma_t"]
q = data["q"]
sigma_a = data["sigma_a"]
sigma_s = data["sigma_s"]
sigma_t_diff = data["sigma_t_diff"]
q_diff = data["q_diff"]
sigma_a_diff = data["sigma_a_diff"]
sigma_s_diff = data["sigma_s_diff"]
image = sigma_s_diff
#image = q_diff
print(np.max(image))
fig = plt.figure(figsize=(8,8));
ax = fig.add_subplot(111)
img_view = ax.imshow(image[:,:], cmap='jet', vmin=np.min(image), vmax=np.max(image), origin='lower')

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(img_view, cax=cax)

plt.show()